# QUBO models from GULP

- [Read the structures](#structures)
- [Build the QUBO matrix](#build_qubo)
    - [Ewald](#ewald)
    - [Buckingham](#buckingham)
    

In [1]:
import numpy as np
import pandas as pd

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.periodic_table import Element
from pymatgen.io.cif import *

from ase.visualize import view


from pymatgen.io.ase import AseAtomsAdaptor
import sys

import re
import shutil as sh


import copy
from sklearn.metrics import mean_squared_error 

#import dataframe_image as dfi

from scipy import constants

import matplotlib.pyplot as plt

import itertools
from itertools import chain

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse


k_b = constants.physical_constants['Boltzmann constant in eV/K'][0]
# print(k_b)
def vview(structure):
    view(AseAtomsAdaptor().get_atoms(structure))

np.seterr(divide='ignore')
plt.style.use('tableau-colorblind10')

import seaborn as sns
from QG_functions import *

# <a id='structures'>Read the structures</a>

# <a id='build_qubo'>Build the QUBO matrix</a>

# <a id='ewald'>Ewald</a>

### Ewald Summation Equations

#### Real Space Summation
The real space part of the Ewald summation for the interaction between particles $i$ and $j$ is given by:

$$
d_{ij}^{\text{real}} = \sum_{\mathbf{n} \neq 0} \frac{\text{erfc}(\alpha |\mathbf{r}_{ij} + \mathbf{n}|)}{|\mathbf{r}_{ij} + \mathbf{n}|} + \frac{\text{erfc}(\alpha |\mathbf{r}_{ij}|)}{|\mathbf{r}_{ij}|}
$$

#### Self-Interaction Correction
The self-interaction correction term for particle $i$ is:

$$
d_{ii}^{\text{self}} = -\frac{\alpha}{\sqrt{\pi}}
$$

#### Reciprocal Space Summation
The reciprocal space part of the Ewald summation for the interaction between particles $i$ and $j$ is given by:

$$
d_{ij}^{\text{reciprocal}} = \sum_{\mathbf{k} \neq 0} \frac{4 \pi}{V |\mathbf{k}|^2} \exp\left(-\frac{|\mathbf{k}|^2}{4 \alpha^2}\right) \cos(\mathbf{k} \cdot \mathbf{r}_{ij})
$$

#### Total Ewald Summation Matrix
The total Ewald summation matrix element $d_{ij}$ is the sum of the real space, reciprocal space, and self-interaction terms:

$$
d_{ij} = d_{ij}^{\text{real}} + d_{ij}^{\text{reciprocal}}
$$

$$
d_{ii} = d_{ii}^{\text{real}} + d_{ii}^{\text{self}}
$$

#### Potential Energy Calculation
The potential energy of the system is calculated as:

$$
E = \frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} q_i q_j d_{ij}
$$


In [9]:
import numpy as np
import math

def compute_ewald_matrix(positions, lattice_vectors, alpha=None, real_depth=5, reciprocal_depth=5):
    """
    Compute the Ewald summation matrix for a system of particles.

    Parameters:
    positions (ndarray): Relative positions of particles (Nx3).
    lattice_vectors (ndarray): Lattice vectors of the unit cell (3x3).
    alpha (float): Ewald parameter controlling the split between real and reciprocal sums. If None, it's calculated.
    real_depth (int): Depth of the real space summation.
    reciprocal_depth (int): Depth of the reciprocal space summation.

    Returns:
    ndarray: Ewald summation matrix (NxN).
    """
    N = len(positions)
    
    # Calculate alpha if not provided
    if alpha is None:
        alpha = 2 / (np.linalg.det(lattice_vectors) ** (1.0 / 3))
    
    # Convert relative positions to absolute positions using lattice vectors
    positions = positions @ lattice_vectors
    
    # Initialize the Ewald summation matrix
    ewald_matrix = np.zeros((N, N))
    
    # Generate real space shifts for neighboring cells
    real_shifts = np.array([np.dot(np.array(shift) - real_depth, lattice_vectors)
                            for shift in np.ndindex(2 * real_depth + 1, 2 * real_depth + 1, 2 * real_depth + 1)
                            if shift != (real_depth, real_depth, real_depth)])
    
    # Real space summation
    for i in range(N):
        for j in range(i, N):
            if i != j:
                r_ij = positions[i] - positions[j]
                d_ij = np.linalg.norm(r_ij)
                ewald_matrix[i, j] += math.erfc(alpha * d_ij) / d_ij
                
                # Include contributions from neighboring cells
                for shift in real_shifts:
                    r_shifted = r_ij + shift
                    d_shifted = np.linalg.norm(r_shifted)
                    ewald_matrix[i, j] += math.erfc(alpha * d_shifted) / d_shifted
    
    # Self-interaction term correction
    for i in range(N):
        ewald_matrix[i, i] -= alpha / math.sqrt(math.pi)
    
    # Generate reciprocal space shifts for the Fourier transform contributions
    reciprocal_vectors = 2 * np.pi * np.linalg.inv(lattice_vectors).T
    reciprocal_shifts = np.array([np.dot(np.array(shift) - reciprocal_depth, reciprocal_vectors)
                                  for shift in np.ndindex(2 * reciprocal_depth + 1, 2 * reciprocal_depth + 1, 2 * reciprocal_depth + 1)
                                  if shift != (reciprocal_depth, reciprocal_depth, reciprocal_depth)])
    
    # Reciprocal space summation
    for i in range(N):
        for j in range(i, N):
            for k in reciprocal_shifts:
                k_norm = np.linalg.norm(k)
                if k_norm > 0:
                    k_dot_r = np.dot(k, positions[j] - positions[i])
                    term = (4 * math.pi / (np.linalg.det(lattice_vectors) * k_norm**2)) * \
                           math.exp(-k_norm**2 / (4 * alpha**2)) * math.cos(k_dot_r)
                    ewald_matrix[i, j] += term
    
    # Convert to electrostatic potential energy (unit conversion)
    ewald_matrix *= 14.399645351950543  # Convert to appropriate units, e.g., eV for electrostatic potential
    
    # Symmetry completion: Ensure the matrix is symmetric
    for i in range(N):
        for j in range(i):
            ewald_matrix[i, j] = ewald_matrix[j, i]
    
    return ewald_matrix

def calculate_potential_energy(ewald_matrix, charges):
    """
    Calculate the potential energy of the system given the Ewald summation matrix and charges.

    Parameters:
    ewald_matrix (ndarray): Ewald summation matrix (NxN).
    charges (ndarray): Charges of the particles (N).

    Returns:
    float: Total potential energy of the system.
    """
#     return 0.5 * np.sum(charges[:, np.newaxis] * charges[np.newaxis, :] * ewald_matrix)
    return  charges[:, np.newaxis] * charges[np.newaxis, :] * ewald_matrix

# Example usage:
positions = np.array([[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]])  # Example positions in fractional coordinates
lattice_vectors = np.eye(3)  # Example lattice vectors (unit cube)

# Compute the Ewald matrix once
ewald_matrix = compute_ewald_matrix(positions, lattice_vectors)

# Different sets of charges
charges_list = [
    np.array([1.0, -1.0]),
    np.array([1.0, 1.0]),
    np.array([-1.0, -1.0])
]

# Calculate potential energy for each set of charges
for charges in charges_list:
    potential_energy = calculate_potential_energy(ewald_matrix, charges)
    print(f"Charges: {charges},\n Potential Energy:\n {potential_energy}")


Charges: [ 1. -1.],
 Potential Energy:
 [[-13.71031325   0.23813855]
 [  0.23813855 -13.71031325]]
Charges: [1. 1.],
 Potential Energy:
 [[-13.71031325  -0.23813855]
 [ -0.23813855 -13.71031325]]
Charges: [-1. -1.],
 Potential Energy:
 [[-13.71031325  -0.23813855]
 [ -0.23813855 -13.71031325]]


# <a id='buckingham'>Buckingham</a>